In [39]:
from PIL import Image
import numpy as np

def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr



In [42]:
path1 = 'results/elic_vtest/label/0001.png'
path2 = 'results/elic_vtest/recon/test8_0002.png'
# 示例用法
# 读取两张图像
img1 = Image.open(path2)
img2 = Image.open(path1)
img1 = img1.convert("RGB")
img2 = img2.convert("RGB")

# 将图像转换为 NumPy 数组
img1 = np.array(img1)
img2 = np.array(img2)
print(img1.shape)
print(img2.shape)
# 确保图像具有相同的尺寸和类型
if img1.shape != img2.shape:
    raise ValueError("输入图像必须具有相同的尺寸和类型")

psnr_value = calculate_psnr(img1, img2)
print(f"PSNR: {psnr_value} dB")


(256, 256, 3)
(256, 256, 3)
PSNR: 32.683454776189684 dB
